In [263]:
def jonny(speculated_interest=8,input_amount=.5,\
          days_maturity=90,days_matured=0,
          liquidity=50,time_stretch=15,\
          gas_fee=0.0019,target_interest=16):
    t = (days_maturity-days_matured) / (365 * time_stretch)
    T = (days_maturity-days_matured) / 365
    y_start = liquidity
    g = .1
    accumulated_interest = input_amount * days_matured/365 * speculated_interest/100
    A = ((input_amount)*(speculated_interest/100)*T-(target_interest/1000)*input_amount*T-gas_fee)/input_amount
    resulting_pt_apy = A/T*100
    x_start=y_start/((1-T*(resulting_pt_apy/100))**(-1/t)-1)
    unit_price = 1-A
    expenditure=input_amount-unit_price*input_amount+gas_fee+accumulated_interest
    received = (input_amount * speculated_interest / 100) * (days_maturity) / 365
    output_amount = (1-A)*input_amount
    fee = (input_amount - output_amount) * g
    output_amount = output_amount+fee
    apy = round(((received / expenditure) - 1) * 365 / (days_maturity - days_matured) * 100, 2)
    # calculate pre-slippage
    pt_apy = (1- output_amount/input_amount)/T * 100 
    y_after=liquidity+input_amount
    x_after=y_after/((1-T*(pt_apy/100))**(-1/t)-1)
    total_supply = x_after
    k = (y_after + total_supply)**(1-t) + x_after**(1-t)
    j = input_amount 
    i = y_after + total_supply - pow(k-pow(x_after + j,1-t),1/(1-t))
    
    y_before=liquidity
    x_before=x_after+output_amount
    pt_apy = (1-(j / i))/T * 100 + (1-1/(pow((y_before+x_before)/x_before,t)))/T
    
    row = []
    row.append(input_amount)
    row.append(pt_apy)
    row.append(unit_price)
    row.append(resulting_pt_apy)
    row.append(expenditure)
    row.append(received)
    row.append(received - expenditure)
    row.append(apy)
    return row

In [234]:
import ipywidgets as widgets
import numpy as np
import pandas as pd

def button_run_on_click_j(_):
    status_label_j.value = "running...."

    columns = ['Input','FYT APY', 'FYT Price', 'Slippage APY', 'Spent', 'Received', 'Gain', 'APY']
    values = []

    for i in np.arange(.5, 5.1, .1):
      results = jonny(speculated_interest=yield_interest_input_j.value, \
                                input_amount = i, \
                                days_maturity = float(maturity_period_input_j.value), \
                                days_matured = float(days_matured_input_j.value),\
                                liquidity = liquidity_input_j.value, \
                                time_stretch = time_stretch_input_j.value, \
                                target_interest=target_interest_input_j.value)
      if results is not None:
        values.append(results)

    df = pd.DataFrame(values, columns = columns)
    pd.options.display.width = 20
    df.style.set_properties(**{'text-align': 'left'})
    
    status_label_j.value = ""

    result_box_j = setup_ui(df)

    main_box_j.children = [yield_interest_input_j, target_interest_input_j, maturity_period_input_j, days_matured_input_j, liquidity_input_j, time_stretch_input_j, button_run_j, status_label_j, result_box_j]


def setup_ui(df):
    out = widgets.Output()
    with out:
        display(df)
    return out
jonny_label = widgets.Label()
jonny_label.value = "Instant Calc"
status_label_j = widgets.Label()
status_label_j.layout.width = '300px'

layout = widgets.Layout(
    align_content='flex-start',
)
style = {
    'description_width':'initial'
}
yield_interest_input_j = widgets.FloatText(description='speculated interest:', value=8, style=style, layout=layout)
target_interest_input_j = widgets.FloatText(description='target interest:', value=16, style=style, layout=layout)
maturity_period_input_j = widgets.Dropdown(description='term length:', options=['90', '120', '180'],style=style, layout=layout)
days_matured_input_j = widgets.FloatText(description='days matured:', value=0, style=style, layout=layout)
liquidity_input_j = widgets.FloatText(description='liquidity:', value=50, style=style, layout=layout)
time_stretch_input_j = widgets.FloatText(description='time stretch:', value=15, style=style, layout=layout)
button_run_j = widgets.Button(description="Run")
main_box_j = widgets.VBox([jonny_label,yield_interest_input_j, target_interest_input_j, maturity_period_input_j, days_matured_input_j, liquidity_input_j, time_stretch_input_j, button_run_j, status_label_j])
button_run_j.on_click(button_run_on_click_j)
display(main_box_j)